In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

baselink = 'https://www.timeout.com/film/best-movies-of-all-time'
MoviesLinks = []

r = requests.get(baselink)
soup = BeautifulSoup(r.content, 'html.parser')

spans = soup.find_all('a', {'data-testid': 'tile-link_testID'})

for x in spans:
    if x!='':
        movielink = 'https://www.timeout.com' + x['href']
        MoviesLinks.append(movielink)

movieslist = []
for links in MoviesLinks:
    r = requests.get(links)
    soup = BeautifulSoup(r.content, 'html.parser')

    title_tag = soup.find("h1", {"class": "_h1_1ucvn_1"})
    title = title_tag.text.strip() if title_tag else 'N/A'

    space1 = soup.find("ul", {"class": "_list_1dabe_42"})
    if space1 and len(space1.contents) >= 3:
        to_rate = space1.contents[0].find_all('span')
        rate = to_rate[1].text.strip() if len(to_rate) > 1 else 'N/A'

        to_date = space1.contents[1].find_all('span')
        date = to_date[1].text.strip().split() if len(to_date) > 1 else ['N/A'] * 4
        day_name = date[0] if len(date) > 0 else 'N/A'
        day_number = date[1] if len(date) > 1 else 'N/A'
        month = date[2] if len(date) > 2 else 'none'
        year = date[3] if len(date) > 3 else 'N/A'

        to_duration = space1.contents[2].find_all('span')
        fullduration = to_duration[1].text.strip().split() if len(to_duration) > 1 else ['N/A']
        duration = fullduration[0] if len(fullduration) > 0 else 'N/A'
    else:
        rate = 'N/A'
        day_name = 'N/A'
        day_number = 'N/A'
        month = 'none'
        year = 'N/A'
        duration = 'N/A'

    space2 = soup.find_all("ul", {"class": "_list_1dabe_42"})
    if len(space2) > 1 and len(space2[1].contents) >= 3:
        to_director = space2[1].contents[0].find_all('span')
        director = to_director[1].text.strip() if len(to_director) > 1 else 'N/A'

        to_screenwriter = space2[1].contents[1].find_all('span')
        screenwriters = to_screenwriter[1].text.strip() if len(to_screenwriter) > 1 else 'N/A'

        to_cast = space2[1].contents[2].find('ul', {'class': '_actor-list_1dabe_64'})
        actors = [li.text.strip() for li in to_cast.find_all('li')] if to_cast else 'N/A'
    else:
        director = 'N/A'
        screenwriters = 'N/A'
        actors = 'N/A'

    data = {
        'title': title,
        'rate': rate,
        'day_name': day_name,
        'day_number': day_number,
        'month': month,
        'year': year,
        'full_date': ' '.join(date) if date else 'N/A',
        'duration': duration,
        'director': director,
        'screenwriter': screenwriters,
        'actors': actors
    }

    movieslist.append(data)

df = pd.DataFrame(movieslist)

df = df.drop_duplicates(subset='title', keep='first')

df.to_excel('excelsheet.xlsx', index=False)

df


,title,rate,day_name,day_number,month,year,full_date,duration,director,screenwriter,actors
0,2001: A Space Odyssey,U,Friday,28,November,2014,Friday 28 November 2014,141,Stanley Kubrick,"Stanley Kubrick, Arthur C Clarke","[William Sylvester, Keir Dullea, Gary Lockwood..."
2,The Godfather,N/A,N/A,N/A,none,N/A,Friday 28 November 2014,N/A,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","[Richard Castellano, Robert Duvall, Marlon Bra..."
4,Citizen Kane,N/A,N/A,N/A,none,N/A,Friday 28 November 2014,N/A,Orson Welles,"Orson Welles, Herman J Mankiewicz","[Agnes Moorehead, Alan Ladd, Ray Collins, Paul..."
6,"Jeanne Dielman, 23 Quai du Commerce, 1080 Brux...",N/A,N/A,N/A,none,N/A,Friday 28 November 2014,N/A,Chantal Anne Akerman,Chantal Anne Akerman,"[Delphine Seyrig, Jan Decorte, Henri Storck, J..."
8,Raiders of the Lost Ark,N/A,N/A,N/A,none,N/A,Friday 28 November 2014,N/A,Steven Spielberg,Lawrence Kasdan,"[Harrison Ford, Karen Allen, Paul Freeman, Ron..."
...,...,...,...,...,...,...,...,...,...,...,...
191,Nashville,N/A,N/A,N/A,none,N/A,Friday 29 June 2012,N/A,Robert Altman,Joan Tewkesbury,"[Ned Beatty, Karen Black, Ronee Blakley, Keith..."
193,Don't Look Now,N/A,N/A,N/A,none,N/A,Friday 29 June 2012,N/A,Nicolas Roeg,"Allan Scott, Chris Bryant","[Renato Scarpa, Donald Sutherland, Julie Chris..."
195,Bonnie and Clyde,N/A,N/A,N/A,none,N/A,Friday 29 June 2012,N/A,Arthur Penn,"David Newman, Robert Benton","[Warren Beatty, Faye Dunaway, Michael J Pollar..."
197,Get Out,15,Friday,17,March,2017,Friday 17 March 2017,103,N/A,N/A,N/A
